In [ ]:
!pip install langchain faiss-cpu PyMuPDF python-docx

In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install sentence-transformers

In [ ]:
from google.colab import files
import fitz  # PyMuPDF
import io
import os
from docx import Document as DocxDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains.question_answering import load_qa_chain
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings

2: Functions for Extracting Text from PDF Files

In [ ]:
# Function to extract text from .pdf files
def extract_text_from_pdf(pdf_bytes):
    pdf_document = fitz.open(stream=pdf_bytes, filetype="pdf")
    text = "".join(page.get_text() for page in pdf_document)
    return text.replace("\n", "")

In [ ]:
# Function to extract text from .docx files
def extract_text_from_docx(docx_bytes):
    docx_document = DocxDocument(io.BytesIO(docx_bytes))
    text = "".join(para.text for para in docx_document.paragraphs)
    return text.replace("\n", "")

In [ ]:
# Function to extract text from .txt files
def extract_text_from_txt(txt_bytes):
    return txt_bytes.decode('utf-8').replace("\n", "")


In [ ]:
# Function to upload documents
def upload_documents():
    uploaded = files.upload()
    documents = {filename: uploaded[filename] for filename in uploaded.keys() if filename.split('.')[-1].lower() in ['pdf', 'docx', 'txt']}
    return documents

# Get text from uploaded documents
documents = upload_documents()
texts = {filename: (extract_text_from_pdf(file_bytes) if filename.endswith('.pdf') else
                    extract_text_from_docx(file_bytes) if filename.endswith('.docx') else
                    extract_text_from_txt(file_bytes))
         for filename, file_bytes in documents.items()}


Saving coal_petro.pdf to coal_petro (1).pdf
Saving covd_text.docx to covd_text (1).docx
Saving light_notes.txt to light_notes (1).txt
Saving sound.pdf to sound (1).pdf


In [ ]:
# Print the extracted texts
for filename, text in texts.items():
     print(f"Contents of {filename}:")
     print(text)

Contents of coal_petro (1).pdf:
COAL AND PETROLEUMCOAL AND PETROLEUMCan air, water and soil be exhaustedby human activities? You have alreadystudied about water in Class VII. Is watera limitless resource?In the light of the availability ofvarious resources in nature, naturalresources can be broadly classifiedinto two kinds:Inexhaustible Natural ResourcesThese resources are present in unlimitedquantity in nature and are not likely tobe exhausted by human activities.Examples are: sunlight, air.Exhaustible Natural ResourcesThe amount of these resources innature is limited. They can beexhausted by human activities.Examples of these resources are forests,wildlife, minerals, coal, petroleum,natural gas etc.Activity 5.2It is a group activityTake some containers. Fill themwith  popcorn/peanuts/roastedgram/toffees. Divide students intogroups of seven each. Further divideeach group into three subgroupscontaining 1, 2 and 4 students.Label them as first, second andthird generation respectively. Th

In [ ]:
# Split text into chunks
chunk_size = 400  # Increased chunk size for larger documents
chunk_overlap = 100  # Moderate overlap to maintain context

splitter = RecursiveCharacterTextSplitter(
    length_function = len,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

chunks = [(doc_name, chunk) for doc_name, text in texts.items() for chunk in splitter.split_text(text)]
docs = [chunk[1] for chunk in chunks]

In [ ]:
print(type(chunks),len(chunks))

<class 'list'> 391


In [ ]:
print(type(docs),len(docs),docs)

<class 'list'> 391 ['COAL AND PETROLEUMCOAL AND PETROLEUMCan air, water and soil be exhaustedby human activities? You have alreadystudied about water in Class VII. Is watera limitless resource?In the light of the availability ofvarious resources in nature, naturalresources can be broadly classifiedinto two kinds:Inexhaustible Natural ResourcesThese resources are present in unlimitedquantity in nature and are not', 'Natural ResourcesThese resources are present in unlimitedquantity in nature and are not likely tobe exhausted by human activities.Examples are: sunlight, air.Exhaustible Natural ResourcesThe amount of these resources innature is limited. They can beexhausted by human activities.Examples of these resources are forests,wildlife, minerals, coal, petroleum,natural gas etc.Activity 5.2It is a group', 'resources are forests,wildlife, minerals, coal, petroleum,natural gas etc.Activity 5.2It is a group activityTake some containers. Fill themwith  popcorn/peanuts/roastedgram/toffees.

In [ ]:
def get_vector_store(text_chunks):
    # Initialize Hugging Face Embeddings
    model_name = "sentence-transformers/all-mpnet-base-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    return vector_store

# Example usage:
text_chunks = docs
vector_store = get_vector_store(text_chunks)

In [ ]:
vector_store

In [ ]:
def answer_question(question, vector_store):
    # Get the top 5 similar documents with their scores
    similar_docs_with_scores = vector_store.similarity_search_with_score(question, k=5)

    # Extract the top document and its score
    top_doc, score = similar_docs_with_scores[0]
    return  top_doc

In [ ]:
question = "tell me About Covid-19"
answers = answer_question(question, vector_store)
answers

Document(page_content='that co-exists with humans for a long time, becomes seasonal and is sustained within human bodies.” [19] Ever since the pandemic started, we have been bombarded daily with a relentless stream of data but, in June 2020, roughly half a year after the beginning of the outbreak, our knowledge is still very patchy and as a result we still don’t really know just how dangerous COVID-19 is. Despite the')